In [1]:
#load metadata for all (secondary) outputs available for this project
import os
import json
objs = []
for entry in os.scandir("input/meta"):
    with open(entry.path, "r") as f:
        obj = json.loads(f.read())

    if obj["removed"] != True:
        objs.append(obj)

print("number of metadata", len(objs))

#load participants columns/info (for phenotype - if you have it defined for your project)
with open("input/participants.json") as f:
    participants = json.load(f)
    
with open("input/participants_column.json") as f:
    participants_column = json.load(f)

number of metadata 717


In [2]:
# EDIT ME
# Now you need to pick and choose which objects # you want to load 
# into this notebook to perform your data analysis.
#
# Itereate through the objs list from above step and pick the objects you need.
# The actual file path for each object is stored in obj["_secondaryPath"]

# import any library you need to load data (for csv, pandas works nicely)
import pandas
import numpy as np
# you probably need to create multiple lists to load various things
myCSVs = {}
sub_list = []
for obj in objs:
    #put whatever the condition you'd like to use
    #if you don't know which field/keys to use, just
    #open up one of the .json under input/meta directory
    #and see what key/values exists
    if obj["datatype"]["name"] == "neuro/parc-stats": 
        subject = obj["meta"]["subject"]
        csvPath = "input/"+obj["_secondaryPath"]+"/parc-stats/parc_nodes.csv"
        myCSVs[subject] = pandas.read_csv(csvPath)
        sub_list.append(subject)

In [3]:
import pandas as pd
rtp = pd.read_csv("rtp_acute_cases.csv",index_col=0)

In [4]:
#IDs of all the cc parts
left_parc_IDs = [251, 252, 253, 254, 255]

#related StuctureIDs
df_ex = myCSVs[sub_list[0]]
left_str_IDs = df_ex[df_ex['parcID'].isin(left_parc_IDs)]['structureID']
left_str_IDs = left_str_IDs.tolist()
left_str_IDs

['CC_Posterior',
 'CC_Mid_Posterior',
 'CC_Central',
 'CC_Mid_Anterior',
 'CC_Anterior']

In [5]:
metrics = ['fa']
left_sc_means = np.zeros((len(sub_list), len(left_parc_IDs), len(metrics)))

In [6]:
for s, sub in enumerate(sub_list):
    df = myCSVs[sub]
    #select some rows
    df_sc_L = df[df['parcID'].isin(left_parc_IDs)]
    #select some columns
    L = df_sc_L[metrics]    
    left_sc_means[s,:,:] = L

In [7]:
!pip install scipy==1.7.0

In [8]:
import scipy.stats

#plot individual cc of subjects with an actual value of rtp
#(remember: 4 subjects of the late recovery group do not have it)
rtp_days = []
for s, sub in enumerate(sub_list):
    if np.isnan(rtp.loc[int(sub),"TIME_TORTP_DAYS"]) == False:
        rtp_days.append(rtp.loc[int(sub),"TIME_TORTP_DAYS"])
    
#cc
r_list_fa = []
p_list_fa = []
for r, region in enumerate(left_parc_IDs):
    for m, metric in enumerate(metrics):
        left_sc_means_tmp = []
        for s, sub in enumerate(sub_list):
            if rtp.loc[int(sub), 'rtpLabel'] == 0:
                left_sc_means_tmp.append(left_sc_means[s,r,m])
            elif rtp.loc[int(sub), 'rtpLabel'] == 1:
                if np.isnan(rtp.loc[int(sub),"TIME_TORTP_DAYS"]) == False:
                    left_sc_means_tmp.append(left_sc_means[s,r,m])
        sl, inter, r_val, p_val, std_err = scipy.stats.linregress(rtp_days, left_sc_means_tmp, alternative='less')
        r_list_fa.append(r_val)
        p_list_fa.append(p_val)

In [9]:
#correction with FDR for FA
ranking = np.argsort(np.argsort(p_list_fa))
p_fdr_corrected = p_list_fa * (ranking+1) / len(left_str_IDs)
for p, p_fdr in enumerate(p_fdr_corrected):
    if p_fdr < 0.001:
        sign = '***'
    elif p_fdr > 0.001 and p_fdr < 0.01:
        sign = '**'
    elif p_fdr > 0.01 and p_fdr < 0.05:
        sign = '*'
    else:
        sign = '(ns)'
    print("%s: r=%.02f, p_fdr=%.05f %s" %(left_str_IDs[p], r_list_fa[p], p_fdr, sign)) 

CC_Posterior: r=-0.11, p_fdr=0.22370 (ns)
CC_Mid_Posterior: r=-0.37, p_fdr=0.00108 **
CC_Central: r=-0.20, p_fdr=0.06732 (ns)
CC_Mid_Anterior: r=-0.28, p_fdr=0.01573 *
CC_Anterior: r=-0.29, p_fdr=0.00983 **


In [10]:
#aggregated results for R in FA
np.mean(r_list_fa)

-0.25190723094346207